In [5]:
import numpy as np
from scipy.optimize import minimize


In [6]:
X = np.random.rand(150,75)
(N,D) = X.shape

In [7]:
theta_1 = np.empty(5)
theta_1.fill(10)

theta_2 = np.empty(5)
theta_2.fill(-10)

theta = np.hstack((theta_1,theta_2))
np.random.shuffle(theta)

theta


array([ 10.,  10., -10., -10., -10., -10., -10.,  10.,  10.,  10.])

In [8]:
theta_rest = np.zeros(65)
theta = np.hstack((theta,theta_rest))
theta

array([ 10.,  10., -10., -10., -10., -10., -10.,  10.,  10.,  10.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [9]:
epsilon = 0.1 * np.random.randn(150) + 0
epsilon

array([  2.20772008e-02,  -6.37181174e-02,   1.12772949e-01,
         7.04520355e-02,  -1.09009356e-02,   4.53616188e-02,
         3.39141233e-02,  -2.36632601e-02,   7.02751405e-03,
        -1.93877208e-01,   1.04924810e-01,  -1.85323356e-01,
        -3.12111428e-02,   1.20967261e-02,  -9.94723659e-02,
        -1.60686100e-01,  -1.49592286e-01,   5.99392257e-03,
        -1.04401145e-03,   2.07658036e-01,   5.02870829e-02,
        -1.84002530e-01,  -4.29084717e-02,  -1.13360733e-02,
         6.73238018e-02,  -1.99029098e-01,   1.69044646e-02,
         9.26291047e-02,  -1.13095361e-01,  -8.14664480e-03,
         4.61161400e-02,   1.30738569e-03,   1.10114611e-02,
         1.08099383e-01,   8.29861092e-02,   2.67390339e-02,
         8.89696859e-02,   2.23557234e-02,   1.07865061e-01,
        -1.97952226e-01,   2.07363677e-02,  -3.30389553e-02,
         6.41832134e-05,   1.23370660e-01,  -1.50741353e-01,
         5.99967560e-02,   1.07690041e-01,   4.90780782e-03,
         1.60535582e-02,

In [10]:
y = np.dot(X,theta) + epsilon
y

array([ -8.4834258 ,  -9.72095104,   7.15223426,  -5.85974116,
        13.97217553,   0.73681238,  -1.14462301,  -1.41830577,
       -18.96994346,  -8.86983564,  -8.95898058,   6.67516784,
        12.94622199,  11.53343747,  -7.5802547 ,   5.01647187,
        13.68231368,   4.02188413,  11.18484557,   4.84552358,
        20.91221873,   4.83371648,   9.56212864,  -8.1587667 ,
       -12.24201188,   1.11357933,  -4.74771802,   2.3084843 ,
        -3.09850053,  22.83466861,  -6.77166634,  15.06165252,
       -11.5136915 ,   3.72707553,  -4.85225902, -18.06571958,
        15.17186328,  14.50294128, -15.24996021,  10.34059218,
         6.08248818,  12.37578358,  -7.29797322,   0.08754828,
        -8.65413257,  12.25630603, -13.96877379,   6.77065696,
       -11.11504354,   0.40737925,  -3.0704786 , -14.31818043,
         7.56918919, -17.90244681,  -8.43344466,  10.98439151,
        -4.74079482,  21.70615219,   6.87566206, -14.26070668,
         5.15584876, -12.91612942,  -0.16551051,   6.04

In [11]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=70, random_state=42)

In [12]:
X_test,X_validation,y_test,y_validation = train_test_split(X_test, y_test, test_size=50, random_state=42)

In [13]:
from scipy.optimize import minimize
lambda_loss = np.zeros((11,2))
lambda_theta = np.zeros((11,D))
theta_learned = np.zeros(D)

def ridge(Lambda):
  def ridge_obj(theta):
    return ((np.linalg.norm(np.dot(X_train,theta) - y_train))**2)/(2*N) + Lambda*(np.linalg.norm(theta))**2
  return ridge_obj

def compute_loss(theta):
  return ((np.linalg.norm(np.dot(X_validation,theta) - y_validation))**2)/(2*N)

for i in range(-5,6):
  Lambda = 10**i;
  theta_opt = minimize(ridge(Lambda), theta)
  lambda_loss[i] = Lambda, compute_loss(theta_opt.x) 
  lambda_theta[i] = theta_opt.x

least_loss_lambda = lambda_loss[np.argmin(lambda_loss,axis=0)[1],[0]]
theta_learned = lambda_theta[np.argmin(lambda_loss,axis=0)[1]]
#get the theta for the least_loss_lambda


All values are non-zero

In [15]:
theta_learned[theta_learned < 10**-3] = 0
theta_learned

array([  1.00791999e+01,   9.67717544e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   9.92444628e+00,   9.81893619e+00,
         9.93268353e+00,   1.85648579e-01,   1.67937478e-02,
         0.00000000e+00,   0.00000000e+00,   9.80102382e-02,
         8.99378534e-02,   0.00000000e+00,   1.78024164e-01,
         1.86708349e-01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   9.71913220e-02,   7.02681095e-02,
         1.91023923e-01,   1.23810873e-01,   1.75223301e-01,
         7.02959394e-02,   1.75584172e-02,   0.00000000e+00,
         0.00000000e+00,   3.75805150e-02,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   4.36521760e-02,
         0.00000000e+00,   1.04866878e-02,   0.00000000e+00,
         5.51773217e-03,   1.61093264e-01,   0.00000000e+00,
         0.00000000e+00,   3.27327473e-02,   2.95101424e-01,
         1.33066329e-01,   0.00000000e+00,   3.20474256e-02,
         1.97271075e-01,

Results after scaling the values to be > 10**-3

In [22]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train) 
clf.decision_function(X_train)

/home/himaja/workspace/csml/venv/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function decision_function is deprecated;  and will be removed in 0.19.
  warnings.warn(msg, category=DeprecationWarning)


array([  0.18883803,   4.84545163,  11.9991468 ,   5.6141756 ,
         0.50217644,   1.62492728,  -9.77975702, -15.5829887 ,
         1.71671529,  -5.52767969,  -8.23387802,   0.33239026,
       -16.84078324,  -6.91791489,   1.65788672,  -2.36749634,
         5.24736771,  -1.39195085,   0.78143092,   8.83954727,
        -5.50729532,   3.29356812, -10.70117245,  14.40485843,
        -7.90702421,  13.33098932, -16.11229878,   9.83261948,
        11.72434576,  11.09884347,  -3.92270298,   4.81247937,
       -14.08275775,  13.22960208, -22.42383295,  -2.02737051,
        -0.52591935,   3.3914939 ,   6.36064956,   5.41576616,
        -7.05943169,  -2.61523355,  -3.83534497,  17.10004971,
       -11.19235955,   2.19116512, -13.84718133,   1.98043826,
        -6.28911737, -12.79997906,   7.75434597,  11.30215334,
         5.15127445,   8.67179488, -10.86802733,  10.99182041,
       -15.87861452,  -0.47318072,   4.84084067,  19.88353145,
         6.78854062, -12.36208725,  14.20548338,   2.11